# Hyperparameter tuning with Cloud ML Engine

**Learning Objectives:**
  * Improve the accuracy of a model by hyperparameter tuning

## Create command-line program

In order to submit to Cloud ML Engine, we need to create a distributed training program. Let's convert our housing example to fit that paradigm, using Experiment and Estimator.

In [1]:
%bash
rm -rf trainer
mkdir trainer
touch trainer/__init__.py

In [2]:
%%writefile trainer/house.py
import os
import math
import json
import shutil
import argparse
import numpy as np
import pandas as pd
import tensorflow as tf

def train(output_dir, batch_size, learning_rate):
  tf.logging.set_verbosity(tf.logging.INFO)
  
  # Read dataset and split into train and eval
  df = pd.read_csv("https://storage.googleapis.com/ml_universities/california_housing_train.csv", sep=",")
  df['num_rooms'] = df['total_rooms'] / df['households']
  msk = np.random.rand(len(df)) < 0.8
  traindf = df[msk]
  evaldf = df[~msk]

  # Train and eval input functions
  SCALE = 100000
  
  train_input_fn = tf.estimator.inputs.pandas_input_fn(x = traindf[["num_rooms"]],
                                                       y = traindf["median_house_value"] / SCALE,  # note the scaling
                                                       num_epochs = 1,
                                                       batch_size = batch_size, # note the batch size
                                                       shuffle = True)
  
  eval_input_fn = tf.estimator.inputs.pandas_input_fn(x = evaldf[["num_rooms"]],
                                                      y = evaldf["median_house_value"] / SCALE,  # note the scaling
                                                      num_epochs = 1,
                                                      batch_size = len(evaldf),
                                                      shuffle=False)
  
  # Define feature columns
  features = [tf.feature_column.numeric_column('num_rooms')]
  
  def train_and_evaluate(output_dir):
    # Compute appropriate number of steps
    num_steps = (len(traindf) / batch_size) / learning_rate  # if learning_rate=0.01, hundred epochs

    # Create custom optimizer
    myopt = tf.train.FtrlOptimizer(learning_rate = learning_rate) # note the learning rate

    # Create rest of the estimator as usual
    estimator = tf.estimator.LinearRegressor(model_dir = output_dir, 
                                             feature_columns = features, 
                                             optimizer = myopt)

    train_spec = tf.estimator.TrainSpec(input_fn = train_input_fn,
                                        max_steps = num_steps)
    eval_spec = tf.estimator.EvalSpec(input_fn = eval_input_fn,
                                      steps = None)
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

  # Run the training
  shutil.rmtree(output_dir, ignore_errors=True) # start fresh each time
  train_and_evaluate(output_dir)
    
if __name__ == '__main__' and "get_ipython" not in dir():
  parser = argparse.ArgumentParser()
  parser.add_argument(
      '--learning_rate',
      type = float, default = 0.01
  )
  parser.add_argument(
      '--batch_size',
      type = int, default = 30
  ),
  parser.add_argument(
      '--output_dir',
      help = 'GCS location to write checkpoints and export models',
      required = True
  )
  parser.add_argument(
      '--job-dir',
      help = 'this model ignores this field, but it is required by gcloud',
      default = 'junk'
  )
  args = parser.parse_args()
  train(args.output_dir, args.batch_size, args.learning_rate)

Writing trainer/house.py


In [3]:
%bash
rm -rf house_trained
gcloud ml-engine local train --module-name=trainer.house --job-dir=output_dir --package-path=$(pwd)/trainer \
   -- --output_dir=house_trained --batch_size=30 --learning_rate=0.02

INFO:tensorflow:TF_CONFIG environment variable: {u'environment': u'cloud', u'cluster': {}, u'job': {u'args': [u'--output_dir=house_trained', u'--batch_size=30', u'--learning_rate=0.02', u'--job-dir', u'output_dir'], u'job_name': u'trainer.house'}, u'task': {}}
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8bcb1ba510>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': 'house_trained', '_save_summary_steps': 100}
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 600 secs (

# Create hyperparam.yaml

In [4]:
%writefile hyperparam.yaml
trainingInput:
  hyperparameters:
    goal: MINIMIZE
    maxTrials: 5
    maxParallelTrials: 1
    hyperparameterMetricTag: loss
    params:
    - parameterName: batch_size
      type: INTEGER
      minValue: 8
      maxValue: 512
      scaleType: UNIT_LOG_SCALE
    - parameterName: learning_rate
      type: DOUBLE
      minValue: 0.001
      maxValue: 0.1
      scaleType: UNIT_LOG_SCALE

Writing hyperparam.yaml


In [ ]:
%bash
OUTDIR=gs://cloud-training-demos-ml/house_trained   # CHANGE bucket name appropriately
gsutil rm -rf $OUTDIR
gcloud ml-engine jobs submit training house_$(date -u +%y%m%d_%H%M%S) --config=hyperparam.yaml \
   --module-name=trainer.house --package-path=$(pwd)/trainer --job-dir=$OUTDIR \
   -- --output_dir=$OUTDIR

In [ ]:
!gcloud ml-engine jobs describe house_170820_034011